# Misctools tutorial

In my experience, bioinformaticians typically do highly situation-specific stuff to their data. So instead of providing entire pipelines that'll never really apply anyway,  I've so made a toolbox of functions you can combine to patch together your own data analysis, UNIX-style.

In [1]:
import os
import misctools

## Fasta sequences

Let's instantiate a fasta sequence manually to work with:

In [2]:
thioredoxin_cds = """ATGGTGAAGCAGATCGAGAGCAAGACTGCTTTTCAGGAAGCCTTGGACGCTGCAGGTG
ATAAACTTGTAGTAGTTGACTTCTCAGCCACGTGGTGTGGGCCTTGCAAAATGATCAAGCCTTTCTTTCATGATGTTGC
TTCAGAGTGTGAAGTCAAATGCATGCCAACATTCCAGTTTTTTAAGAAGGGACAAAAGGTGGGTGAATTTTCTGGAGCC
AATAAGGAAAAGCTTGAAGCCACCATTAATGAATTAGTCTAA""".replace('\n', '')

thioredoxin = misctools.FastaEntry("TXN", thioredoxin_cds)
thioredoxin

<FastaEntry TXN>

---

This object has two main attributes, the header and the sequence.

The header is just a string, while the sequence is a bytearray:

---

In [3]:
thioredoxin.header, thioredoxin.sequence

('TXN',
 bytearray(b'ATGGTGAAGCAGATCGAGAGCAAGACTGCTTTTCAGGAAGCCTTGGACGCTGCAGGTGATAAACTTGTAGTAGTTGACTTCTCAGCCACGTGGTGTGGGCCTTGCAAAATGATCAAGCCTTTCTTTCATGATGTTGCTTCAGAGTGTGAAGTCAAATGCATGCCAACATTCCAGTTTTTTAAGAAGGGACAAAAGGTGGGTGAATTTTCTGGAGCCAATAAGGAAAAGCTTGAAGCCACCATTAATGAATTAGTCTAA'))

---
This makes the object fast and memory efficient.

You can ask its length and slice into it directly:

---

In [4]:
print(len(thioredoxin))
print(thioredoxin[18:54]) # This is a bytearray, not a Fasta Entry!
print(thioredoxin[50]) # This is a byte - i.e. a number between 0 and 255

258
bytearray(b'AGCAAGACTGCTTTTCAGGAAGCCTTGGACGCTGCA')
84


---
The string representation (`str(thioredoxin)`) is fasta-formatted, with a single line for the sequence. If you want to represent it with the sequence over multiple lines, use the `thioredoxin.format` method:

---

In [5]:
print(str(thioredoxin))
print(thioredoxin.format(width=79))

>TXN
ATGGTGAAGCAGATCGAGAGCAAGACTGCTTTTCAGGAAGCCTTGGACGCTGCAGGTGATAAACTTGTAGTAGTTGACTTCTCAGCCACGTGGTGTGGGCCTTGCAAAATGATCAAGCCTTTCTTTCATGATGTTGCTTCAGAGTGTGAAGTCAAATGCATGCCAACATTCCAGTTTTTTAAGAAGGGACAAAAGGTGGGTGAATTTTCTGGAGCCAATAAGGAAAAGCTTGAAGCCACCATTAATGAATTAGTCTAA
>TXN
ATGGTGAAGCAGATCGAGAGCAAGACTGCTTTTCAGGAAGCCTTGGACGCTGCAGGTGATAAACTTGTAGTAGTTGACT
TCTCAGCCACGTGGTGTGGGCCTTGCAAAATGATCAAGCCTTTCTTTCATGATGTTGCTTCAGAGTGTGAAGTCAAATG
CATGCCAACATTCCAGTTTTTTAAGAAGGGACAAAAGGTGGGTGAATTTTCTGGAGCCAATAAGGAAAAGCTTGAAGCC
ACCATTAATGAATTAGTCTAA


---
You can also create reverse complemented and translated versions of the entry:


---

In [6]:
rvc = thioredoxin.reversecomplemented()
trn = thioredoxin.translated(endatstop=False)

print(rvc)
print(trn) # Notice the * at the end because I passed endatstop=False

>TXN
TTAGACTAATTCATTAATGGTGGCTTCAAGCTTTTCCTTATTGGCTCCAGAAAATTCACCCACCTTTTGTCCCTTCTTAAAAAACTGGAATGTTGGCATGCATTTGACTTCACACTCTGAAGCAACATCATGAAAGAAAGGCTTGATCATTTTGCAAGGCCCACACCACGTGGCTGAGAAGTCAACTACTACAAGTTTATCACCTGCAGCGTCCAAGGCTTCCTGAAAAGCAGTCTTGCTCTCGATCTGCTTCACCAT
>TXN
MVKQIESKTAFQEALDAAGDKLVVVDFSATWCGPCKMIKPFFHDVASECEVKCMPTFQFFKKGQKVGEFSGANKEKLEATINELV*


---
Normally we don't instantiate fasta entries manually. Instead, we work with fasta files.

## Fasta files

There are two functions for dealing with fasta files:

`iterfasta` - Read fasta files as FastaEntry objects. Use this as the default.

`simple_iterfasta` - Read fasta files as (header, sequence) tuples.

Both these functions are used the same ways. Each of them are used on an open file to give a generator. While the file is opened, the generator can be iterated over to yield entries:

In [7]:
with open('test/test.fasta') as file:
    entries = misctools.iterfasta(file)
    first_entry = next(entries)
    next_ten_entries = [next(entries) for i in range(10)]
    rest_of_entries = list(entries)
    
first_entry

<FastaEntry Q5ZMT0 Eukaryota;Metazoa;Chordata;Craniata;Vertebrata;Euteleostomi;Archelosauria;Archosauria;Dinosauria;Saurischia;Theropoda;Coelurosauria;Aves;Neognathae;Galloanserae;Galliformes;Phasianidae;Phasianinae;Gallus>

---
There is also the similar `simple_fastaiter`, which simpy yields tuples of strings (header, sequence) as shown below.

---

In [8]:
with open('test/test.fasta') as file:
    entries = misctools.simple_iterfasta(file)
    first_header, first_sequence = next(entries)
    
first_header, first_sequence

('Q5ZMT0 Eukaryota;Metazoa;Chordata;Craniata;Vertebrata;Euteleostomi;Archelosauria;Archosauria;Dinosauria;Saurischia;Theropoda;Coelurosauria;Aves;Neognathae;Galloanserae;Galliformes;Phasianidae;Phasianinae;Gallus',
 'ATCGGTCCATCGTGATTATCGGCCGAGAGACAACCTGAAAACCGACCTCGTATAGCACTTGCCGGTAGATCACAATCCAGGGCCACTAACTAATGACTCTAAAACTTAGACAAGTGGTATCACATACATAATGGGTATTCTGGTTATAAATGGAGCTCCCGTGCATGCATGCGTCGTGTACAATTATAAGTGCGTCCCCCAAGAACCGGGGAGGACTCCTTGACATTATCAGTCCTCATTCTAGGCGATGATCCTGTTGGCCGC')

---
These readers are fast, and only loads the sequences into memory as they are being iterated over. You can safely open that 200 GB fasta file! As always in Python, make sure to use the `with open` syntax to automatically close files.

They only do the most rudementary format checking, though, so make sure the file is actually fasta formatted. 

## Working with kmers

Currently, there are just a few functions and methods available for working with kmers in misctools.

All kmer methods are implemented in Cython and are therefore fast and efficient.

`FastaEntry` has a few methods:

`FastaEntry.kmercounts` counts the number of all the different kmers in an entry. For example, for k=4, there are 4 ^ 4 = 256 kmers, so it returns a 256-length array. The kmers are listed alphabetically. The first number, 3, means that AAAA is present thrice, while the last number, 5, means that TTTT is present 5 times.

In [9]:
thioredoxin.kmercounts(k=4)

array('i', [3, 1, 2, 2, 1, 0, 0, 1, 2, 5, 3, 1, 1, 0, 3, 2, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 2, 1, 1, 2, 1, 2, 5, 0, 1, 2, 0, 1, 2, 1, 2, 1, 2, 2, 0, 0, 0, 1, 0, 1, 0, 3, 2, 1, 1, 2, 1, 0, 1, 3, 1, 2, 1, 0, 1, 1, 0, 2, 1, 2, 1, 0, 0, 2, 2, 2, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 2, 1, 0, 0, 2, 4, 3, 1, 0, 5, 2, 1, 0, 1, 2, 1, 2, 0, 1, 1, 2, 1, 0, 2, 0, 2, 1, 4, 0, 0, 3, 0, 0, 0, 0, 0, 0, 1, 3, 2, 2, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 5, 0, 0, 0, 2, 0, 1, 0, 0, 1, 4, 0, 3, 2, 0, 0, 2, 1, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 2, 0, 3, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 5, 1, 0, 3, 3, 1, 0, 2, 2, 0, 2, 2, 1, 0, 2, 1, 2, 0, 1, 0, 3, 1, 0, 3, 2, 2, 1, 1, 1, 4, 0, 5])

`FastaEntry.fourmer_freq` counts the frequencies of all the 136 possible fourmers if the reading direction is unknown (e.g. TTTT and AAAA is the same).

Note that due to each fourmer being rounded with only 32 bits, it doesn't necessarily sum to exactly one:

In [10]:
frequencies = thioredoxin.fourmer_freq()

print(frequencies[:5])

print(sum(frequencies))

array('f', [0.0313725508749485, 0.007843137718737125, 0.019607843831181526, 0.011764707043766975, 0.007843137718737125])
1.000000060070306


Let's have a look at how to print to files memory-effectively:

## Stream printers

A stream printer allows you to print an iterator to a file a number of objects at a time. This strikes a balance between limiting disk access, which is slow, and keeping the output in memory, which may cause a big memory footprint. There are two stream printer function. The first one, `streamprint`, is used with open files.

In the code above, we got a few fasta entries. Let's print the rest of the entries to a file, 100 entries at a time:

---

In [9]:
os.mkdir('outputs')

with open('outputs/last_entries.fasta', 'w') as file:
    misctools.streamprint(rest_of_entries, file, bufferlength=100)

---
We can combine a fasta reader with a stream printer to read a file, filter it, and write the filtered entries back using very little memory.

Make sure the filtering is done with a generator expression, though, a list would keep all the entries in memory!

---

In [10]:
with open('test/test.fasta') as fastafile, open('outputs/filtered.fasta', 'w') as outfile:
    entries = misctools.iterfasta(fastafile)
    filtered = (entry.translated() for entry in entries if len(entry) >= 150) # generator expression!
    misctools.streamprint(filtered, outfile)

---
The other stream printer function is the fork printer. This allows you to print to several files memory effeciently. Please be aware though, that it keeps a buffer _per file_, so if you make the buffers hold hundreds of megabytes each, problems can arise.

Say you want to iterate over a fasta file and split it into two files, one with sequences from _Bacteria_ and one with sequences from eukaryotes and archaea:
___

In [11]:
with open('test/test.fasta') as fastafile:
    entries = misctools.iterfasta(fastafile)
    
    # Yield (index, object), to print to the index'th filename given.
    generator = ((0, entry) if 'Bacteria' in entry.header else (1, entry) for entry in entries)
    misctools.forkprint(generator, 'outputs/bacteria.fasta', 'outputs/neomura.fasta')

## Trying together iterfasta, streamprint and FastaEntry:

Look through a fasta file for sequences from bacteria. All the ones with a name ending in an uneven number must be reverse complemented. Then translate to amino acids and truncate each of the sequences to before their first stop. Then write to a new file.

No problem!

---

In [15]:
# Create a generator of lines which does processing on the fly.
# Barely any memory footprint!
def process(entries):
    for entry in entries:
        # Skip non-bacteria
        if 'Bacteria;' not in entry.header:
            continue

        # Reverse-complement the ones with uneven numbers in accession
        accession = entry.header.split()[0]
        if int(accession[-1]) % 2 == 1:
            entry = entry.reversecomplemented()

        entry = entry.translated(endatstop=True)

        # A sequence with only a stop codon will return None
        if entry is not None:
            yield entry
                
# Stream it to the output file
with open('test/test.fasta') as fastafile, open('outputs/fixed.fasta', 'w') as outfile:
    generator = process(misctools.iterfasta(fastafile))
    misctools.streamprint(generator, outfile)

---
## Opening files which may be gzipped

The Opener class can be used instead of the `open` function if you don't know if the file will be gzipped or not.
The class reads the first few bytes upon opening to check the signature. If it's that of a gzipped file, it returns a map object that yields unzipped, decoded lines. If not, it works as an ordinary `open`:

---

In [11]:
with misctools.Reader('test/test.fasta') as lines:
    print(next(lines))
    print(next(lines))

>Q5ZMT0 Eukaryota;Metazoa;Chordata;Craniata;Vertebrata;Euteleostomi;Archelosauria;Archosauria;Dinosauria;Saurischia;Theropoda;Coelurosauria;Aves;Neognathae;Galloanserae;Galliformes;Phasianidae;Phasianinae;Gallus

ATCGGTCCATCGTGATTATCGGCCGAGAGACAACCTGAAAACCGACCTCGTATAGCACTTGCCGGTAGATCACAATCCAGGGCCACTAACTAATGACTCTAAAACTTAGACAAGTGGTATCACATACATAATGGGTATTCTGGTTATAAATGGAGCTCCCGTGCATGCATGCGTCGTGTACAATTATAAGTGCGTCCCCCAAGAACCGGGGAGGACTCCTTGACATTATCAGTCCTCATTCTAGGCGATGATCCTGTTGGCCGC



In [12]:
with misctools.Reader('test/gzipped.gz') as lines:
    print(next(lines))
    print(next(lines))

>Q5ZMT0 Eukaryota;Metazoa;Chordata;Craniata;Vertebrata;Euteleostomi;Archelosauria;Archosauria;Dinosauria;Saurischia;Theropoda;Coelurosauria;Aves;Neognathae;Galloanserae;Galliformes;Phasianidae;Phasianinae;Gallus

ATCGGTCCATCGTGATTATCGGCCGAGAGACAACCTGAAAACCGACCTCGTATAGCACTTGCCGGTAGATCACAATCCAGGGCCACTAACTAATGACTCTAAAACTTAGACAAGTGGTATCACATACATAATGGGTATTCTGGTTATAAATGGAGCTCCCGTGCATGCATGCGTCGTGTACAATTATAAGTGCGTCCCCCAAGAACCGGGGAGGACTCCTTGACATTATCAGTCCTCATTCTAGGCGATGATCCTGTTGGCCGC



In [13]:
with misctools.Reader('test/gzipped.gz', 'rb') as lines:
    print(next(lines))
    print(next(lines))

b'>Q5ZMT0 Eukaryota;Metazoa;Chordata;Craniata;Vertebrata;Euteleostomi;Archelosauria;Archosauria;Dinosauria;Saurischia;Theropoda;Coelurosauria;Aves;Neognathae;Galloanserae;Galliformes;Phasianidae;Phasianinae;Gallus\n'
b'ATCGGTCCATCGTGATTATCGGCCGAGAGACAACCTGAAAACCGACCTCGTATAGCACTTGCCGGTAGATCACAATCCAGGGCCACTAACTAATGACTCTAAAACTTAGACAAGTGGTATCACATACATAATGGGTATTCTGGTTATAAATGGAGCTCCCGTGCATGCATGCGTCGTGTACAATTATAAGTGCGTCCCCCAAGAACCGGGGAGGACTCCTTGACATTATCAGTCCTCATTCTAGGCGATGATCCTGTTGGCCGC\n'


## Rounding to significant figures

The `significant` function simply returns string representing the input number, rounded to a given number of significant figures, 3 by default.

In [14]:
numbers = [32352, 11.543647, 1/3, -11.222, 5]

for number in numbers:
    print('Before:', number, 'After:', misctools.significant(number, 4))

Before: 32352 After: 32350
Before: 11.543647 After: 11.54
Before: 0.3333333333333333 After: 0.333
Before: -11.222 After: -11.22
Before: 5 After: 5.000


---
## Assembly statistics
   
`assemblystats` takes three arguments. The first is the input filename. The next two
arguments are related to the `sizes` attribute - that is, the assembly sizes at different contig size cutoffs.
For example, to see assembly sizes from 0 to 20 kbp with a step of 2500, do:

---

In [24]:
step = 2500
xmax = 20000
assemblystats = numpytools.assemblystats('test/assemblyinput', xmax=xmax, step=step)

print('Number of contigs:', assemblystats.ncontigs)
print('Smallest contig:', assemblystats.smallest)
print('Largest contig:', assemblystats.largest)
print('N50:', assemblystats.n50)
print('Assembly size:', assemblystats.size)
print('Sizes:')
for n, size in enumerate(assemblystats.sizes):
    print('\t >=', n*step, 'bp:', size)

Number of contigs: 975492
Smallest contig: 250
Largest contig: 112828
N50: 1552
Assembly size: 891411678
Sizes:
	 >= 0 bp: 891411678
	 >= 2500 bp: 343976349
	 >= 5000 bp: 213200795
	 >= 7500 bp: 145077316
	 >= 10000 bp: 104495922
	 >= 12500 bp: 75206234
	 >= 15000 bp: 55686814
	 >= 17500 bp: 42019796
	 >= 20000 bp: 32483707


In [15]:
# Clean up
for directory, __, files in os.walk('outputs/'):
    for file in files:
        os.remove(os.path.join(directory, file))
os.rmdir('outputs/')

FileNotFoundError: [Errno 2] No such file or directory: 'outputs/'